In [ ]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = ' \
 --conf spark.jars.packages=\
io.projectglow:glow-spark3_2.12:1.1.2,\
io.delta:delta-core_2.12:1.0.1,\
org.apache.hadoop:hadoop-aws:3.2.0, \
 --conf spark.hadoop.io.compression.codecs=io.projectglow.sql.util.BGZFCodec \
 --conf spark.serializer=org.apache.spark.serializer.KryoSerializer \
 pyspark-shell '

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()#

spark
sc = spark.sparkContext

sc

In [ ]:
spark

## GTExのsQTLデータを扱う

In [ ]:
sqtl_parquet="hdfs://[Path to Your sQTL data ]/GTEx_Analysis_v8_QTLs-GTEx_Analysis_v8_sQTL_all_associations-Adipose_Subcutaneous.v8.cis_sqtl.all_pairs.chr22.parquet"

In [ ]:
gtex_sqtl_parquet = spark.read.parquet(sqtl_parquet)

In [ ]:
gtex_sqtl_parquet.rdd.getNumPartitions()

In [ ]:
gtex_sqtl_parquet.count()

In [ ]:
gtex_sqtl_parquet.printSchema()

In [ ]:
gtex_sqtl_parquet.show(truncate=False)

In [ ]:
gtex_sqtl_parquet.select("phenotype_id").distinct().show(truncate=False)

In [ ]:
gtex_sqtl_parquet.select("phenotype_id").distinct().count()

In [ ]:
gtex_sqtl_parquet.select("phenotype_id").count()

In [ ]:
from pyspark.sql.functions import *

gtex_sqtl_bronze = gtex_sqtl_parquet.\
    withColumn("contigName",split(col("variant_id"), "_").getItem(0)).\
    withColumn("start",split(col("variant_id"), "_").getItem(1)).\
    withColumn("referenceAllele",split(col("variant_id"), "_").getItem(2)).\
    withColumn("alternateAlleles",split(col("variant_id"), "_").getItem(3)).\
    select(regexp_replace(col("contigName"),"chr","").alias("contigName"),"start","referenceAllele","alternateAlleles","phenotype_id","variant_id","tss_distance",\
          "af","ma_samples","ma_count","pval_nominal","slope","slope_se")

gtex_sqtl_bronze.show(truncate=False)

In [ ]:
gtex_sqtl_bronze.printSchema()

```
from Hail
ht.show()


+---------------+------------+---------------------------+
| locus         | alleles    | phenotype_id.intron       |
+---------------+------------+---------------------------+
| locus<GRCh38> | array<str> | interval<locus<GRCh38>>   |
+---------------+------------+---------------------------+
| chr1:13550    | ["G","A"]  | [chr1:14829-chr1:14970)   |
| chr1:13550    | ["G","A"]  | [chr1:15038-chr1:15796)   |
| chr1:13550    | ["G","A"]  | [chr1:15947-chr1:16607)   |
| chr1:13550    | ["G","A"]  | [chr1:17055-chr1:17233)   |
| chr1:13550    | ["G","A"]  | [chr1:17368-chr1:17606)   |
| chr1:13550    | ["G","A"]  | [chr1:17742-chr1:17915)   |
| chr1:13550    | ["G","A"]  | [chr1:18061-chr1:18268)   |
| chr1:13550    | ["G","A"]  | [chr1:18366-chr1:24738)   |
| chr1:13550    | ["G","A"]  | [chr1:188584-chr1:188791) |
| chr1:13550    | ["G","A"]  | [chr1:188266-chr1:188439) |
+---------------+------------+---------------------------+

+----------------------+----------------------+--------------+------------+
| phenotype_id.cluster | phenotype_id.gene_id | tss_distance | ma_samples |
+----------------------+----------------------+--------------+------------+
| str                  | str                  |        int32 |      int32 |
+----------------------+----------------------+--------------+------------+
| "clu_51865"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51865"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51868"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51870"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51870"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51870"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51871"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51871"          | "ENSG00000227232.5"  |       -16003 |         18 |
| "clu_51874"          | "ENSG00000279457.4"  |      -181861 |         18 |
| "clu_51873"          | "ENSG00000279457.4"  |      -181861 |         18 |
+----------------------+----------------------+--------------+------------+

+----------+----------+--------------+-----------+----------+
| ma_count |      maf | pval_nominal |     slope | slope_se |
+----------+----------+--------------+-----------+----------+
|    int32 |  float64 |      float64 |   float64 |  float64 |
+----------+----------+--------------+-----------+----------+
|       18 | 1.55e-02 |     6.97e-01 |  1.10e-01 | 2.81e-01 |
|       18 | 1.55e-02 |     5.53e-01 | -1.61e-01 | 2.72e-01 |
|       18 | 1.55e-02 |     3.88e-01 | -2.28e-01 | 2.64e-01 |
|       18 | 1.55e-02 |     4.35e-01 |  1.87e-01 | 2.40e-01 |
|       18 | 1.55e-02 |     4.81e-02 | -5.00e-01 | 2.53e-01 |
|       18 | 1.55e-02 |     3.84e-01 |  2.21e-01 | 2.53e-01 |
|       18 | 1.55e-02 |     6.91e-02 | -4.06e-01 | 2.23e-01 |
|       18 | 1.55e-02 |     1.67e-01 |  3.41e-01 | 2.46e-01 |
|       18 | 1.55e-02 |     4.66e-01 | -1.90e-01 | 2.61e-01 |
|       18 | 1.55e-02 |     5.04e-01 |  1.71e-01 | 2.56e-01 |
+----------+----------+--------------+-----------+----------+
showing top 10 rows
```

```
from hail

gtexadrenal.describe()
----------------------------------------
Global fields:
    'metadata': struct {
        name: str, 
        version: str, 
        reference_genome: str, 
        n_rows: int32, 
        n_partitions: int32
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'phenotype_id': struct {
        intron: interval<locus<GRCh38>>, 
        cluster: str, 
        gene_id: str
    } 
    'tss_distance': int32 
    'ma_samples': int32 
    'ma_count': int32 
    'maf': float64 
    'pval_nominal': float64 
    'slope': float64 
    'slope_se': float64 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------
```

```
chr22:15785057:15787172:clu_28292:ENSG00000206195.10
```
```
phenotype_id.intron   interval<locus<GRCh38>>
[chr1:14829-chr1:14970) 
chr22:15785057:15787172

phenotype_id.cluster  str
clu_51870
clu_28292

phenotype_id.gene_id  str
ENSG00000227232.5
ENSG00000206195.10
```